In [1]:
import os
import sys
import math
#import itertools
import datetime
import numpy as np
import pandas as pd

import keras

Using TensorFlow backend.


In [2]:
# Append TSPerf path to sys.path
nb_dir = os.path.split(os.getcwd())[0]
tsperf_dir = os.path.dirname(os.path.dirname(os.path.dirname(nb_dir)))
if tsperf_dir not in sys.path:
    sys.path.append(tsperf_dir)

from common.metrics import MAPE
import retail_sales.OrangeJuice_Pt_3Weeks_Weekly.common.benchmark_settings as bs

In [3]:
# Data paths
DATA_DIR = '../../data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

In [14]:
def df_from_cartesian_product(dict_in):
    """Generate a Pandas dataframe from Cartesian product of lists.
    
    Args: 
        dict_in (Dictionary): Dictionary containing multiple lists
        
    Returns:
        df (Dataframe): Dataframe corresponding to the Caresian product of the lists
    """
    from collections import OrderedDict
    from itertools import product
    od = OrderedDict(sorted(dict_in.items()))
    cart = list(product(*od.values()))
    df = pd.DataFrame(cart, columns=od.keys())
    return df

def gen_sequence(df, seq_length, seq_cols):
    """Reshape features into an array of dimension (time steps, features)  
    """
    data_array = df[seq_cols].values
    total_timesteps = df.shape[0]
    for start, stop in zip(range(0, total_timesteps-seq_length), range(seq_length, total_timesteps)):
        yield data_array[start:stop, :]

In [5]:
r = 0
print('---- Round ' + str(r+1) + ' ----')
train_df = pd.read_csv(os.path.join(TRAIN_DIR, 'train_round_'+str(r+1)+'.csv'))
train_df['move'] = train_df['logmove'].apply(lambda x: round(math.exp(x)))
train_df.drop('logmove', axis=1, inplace=True)
print(train_df.head(3))
print('')
# Fill missing values
store_list = train_df['store'].unique()
brand_list = train_df['brand'].unique()
week_list = range(bs.TRAIN_START_WEEK, bs.TEST_END_WEEK_LIST[r]+1)
d = {'store': store_list,
     'brand': brand_list,
     'week': week_list}        
data_grid = df_from_cartesian_product(d)
data_filled = pd.merge(data_grid, train_df, how='left', 
                        on=['store', 'brand', 'week'])
print('Number of missing rows is {}'.format(data_filled[data_filled.isnull().any(axis=1)].shape[0]))
print('')
data_filled = data_filled.groupby(['store', 'brand']). \
                          apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
print(data_filled.head(3))
print('')

---- Round 1 ----
   store  brand  week  constant    price1    price2    price3    price4  \
0      2      1    40         1  0.060469  0.060497  0.042031  0.029531   
1      2      1    46         1  0.060469  0.060312  0.045156  0.046719   
2      2      1    47         1  0.060469  0.060312  0.045156  0.046719   

     price5    price6    price7    price8    price9   price10   price11  deal  \
0  0.049531  0.053021  0.038906  0.041406  0.028906  0.024844  0.038984     1   
1  0.049531  0.047813  0.045781  0.027969  0.042969  0.042031  0.038984     0   
2  0.037344  0.053021  0.045781  0.041406  0.048125  0.032656  0.038984     0   

   feat     profit  move  
0   0.0  37.992326  8256  
1   0.0  30.126667  6144  
2   0.0  30.000000  3840  

Number of missing rows is 6204

   brand  store  week  constant    price1    price2    price3    price4  \
0      1      2    40       1.0  0.060469  0.060497  0.042031  0.029531   
1      1      2    41       1.0  0.060469  0.060497  0.042031  0.

In [ ]:
print(data_filled)

In [20]:
data_sub = data_filled[(data_filled.brand==1) & (data_filled.store==2)]
sequence_length = 8
sequence_cols = ['move']
# seq_gen = (list(gen_sequence(train_df[(train_df['store']==cur_store) & (train_df['brand']==cur_brand)], sequence_length, sequence_cols)) 
#            for cur_store, cur_brand in zip(train_df['store'].unique(), train_df['brand'].unique()))
seq_gen = (list(gen_sequence(data_sub[(data_sub['store']==cur_store) & (data_sub['brand']==cur_brand)], sequence_length, sequence_cols)) 
           for cur_store, cur_brand in zip(data_sub['store'].unique(), data_sub['brand'].unique()))
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(91, 8, 1)

In [19]:
list(gen_sequence(data_sub, sequence_length, sequence_cols))

91

In [ ]:
# Model definition
n_steps = 40
n_filters = 32
from keras.models import Sequential, Model
from keras.layers import * #Conv1D, Dense
#sales_in = Input(shape=(n_steps, 1))
#c1 = Conv1D(n_filters, 2, dilation_rate=1, padding='causal', activation='relu')(sales_in)
#c1 = Flatten()(c1)
#output = Dense(2, activation='relu')(c1)
model = Sequential([
    Conv1D(n_filters, 2, dilation_rate=2, padding='causal', activation='relu', input_shape=(n_steps,1)),
    Flatten(),
    Dense(2, activation='relu')
])
model.summary()